In [74]:
import numpy as np
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
import string

%matplotlib inline

In [115]:
letter_string = string.ascii_lowercase

with open('data/letter.txt') as f:
    words_x, words_y, words_letters = [], [], []
    letters = []
    vectors = []
    folds = []
    ys = []
    for line in f:
        entries = line.split('\t')[:-1]
        ys.append(letter_string.index(entries[1]))
        letters.append(entries[1])
        vectors.append(np.array(entries[6:]).astype(int))
        if int(entries[2]) == -1:
            words_y.append(np.array(ys))
            words_letters.append(deepcopy(letters))
            letters = []
            folds.append(entries[5])
            words_x.append(np.array(vectors))
            vectors = []
            ys = []
folds = np.array(folds).astype(int).tolist()

In [109]:
def pad(x, y):
    maxlen = np.max([seq.shape[0] for seq in x])
    D = x[0].shape[1]
    padded_x = []
    padded_y = []
    seq_lens = []
    for seq, labels in list(zip(x, y)):
        seq_len = seq.shape[0]
        padded_seq = np.concatenate([seq, np.zeros((maxlen - seq_len, D))])[None, :, :]
        padded_labels = np.concatenate([labels, np.zeros((maxlen - seq_len))])[None, :]
        padded_x.append(padded_seq)
        padded_y.append(padded_labels)
        seq_lens.append(seq_len)
    return np.concatenate(padded_x), np.concatenate(padded_y), np.array(seq_lens)

def make_classification_data(x, y):
    x_new, y_new = [], []
    for seq, labels in list(zip(x, y)):
        x_new.append(seq)
        y_new.append(labels)
    x_new = np.concatenate(x_new)
    y_new = np.concatenate(y_new)
    return x_new, y_new

In [116]:
x_tr, x_te, y_tr, y_te = [], [], [], []
for fold, x, y in list(zip(folds, words_x, words_y)):
    if fold == 9:
        x_te.append(x)
        y_te.append(y)
    else:
        x_tr.append(x)
        y_tr.append(y)

## Classification Data

In [119]:
x_tr_class, y_tr_class = make_classification_data(x_tr, y_tr)
x_te_class, y_te_class = make_classification_data(x_te, y_te)

In [123]:
np.save('data_class/x_tr', x_tr_class)
np.save('data_class/x_te', x_te_class)
np.save('data_class/y_tr', y_tr_class)
np.save('data_class/y_te', y_te_class)

## Struct Data

In [124]:
x_struct, y_struct, seq_lens = pad(words_x, words_y)

In [141]:
x_tr_struct = x_struct[np.array(folds) != 9]
x_te_struct = x_struct[np.array(folds) == 9]
y_tr_struct = y_struct[np.array(folds) != 9]
y_te_struct = y_struct[np.array(folds) == 9]
seq_lens_tr = seq_lens[np.array(folds) != 9]
seq_lens_te = seq_lens[np.array(folds) == 9]

In [144]:
np.save('data_struct/x_tr', x_tr_struct)
np.save('data_struct/x_te', x_te_struct)
np.save('data_struct/y_tr', y_tr_struct)
np.save('data_struct/y_te', y_te_struct)
np.save('data_struct/seq_lens_tr', seq_lens_tr)
np.save('data_struct/seq_lens_te', seq_lens_te)